In [ ]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold

import gc

from tqdm import tqdm_notebook, tqdm

import json
from typing import NamedTuple

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


import warnings
warnings.filterwarnings(action='ignore')


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
print(torch.__version__)


# from tools import eval_summary, save_feature_importance, merge_preds


device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device

In [ ]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)

df_train.info()

fea_cols = [c for c in df_train.columns if 'layer_' not in c]
layer_cols = [c for c in df_train.columns if 'layer_' in c]

df_train

In [ ]:
import matplotlib.pyplot as plt

def figure_to_array(fig):
    """
    plt.figure를 RGBA로 변환(layer가 4개)
    shape: height, width, layer
    """
    fig.canvas.draw()
    return np.array(fig.canvas.renderer._renderer)

def fea_to_figure(d, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(d, color='black', antialiased=False, linewidth=1)
    ax.set_xlim(0, 225)
    ax.set_ylim(-0.02, 1.0)
    fig.patch.set_visible(False)
    ax.axis('off')
    plt.close()
    
    return fig

def fea_to_array(df):
    arr = []
    for d in df[fea_cols].values:
        fig = fea_to_figure(d)
        arr.append(figure_to_array(fig)[:, :, 1])
        
    df['arr'] = arr
    return df[layer_cols + ['arr']]

from multiprocessing import Pool


dfs = []
step = 10000
params = [df_train[i:i+step] for i in range(0, len(df_train), step)]
# params

with Pool(16) as p:
    dfs = p.map(fea_to_array, params)

In [ ]:
dfs

In [ ]:
cond = (df_train['layer_1'] == 10) & (df_train['layer_2'] == 10) & (df_train['layer_3'] == 10)

In [ ]:
y_cols = [str(i) for i in range(10)]

y_cols = [str(i) for i in range(0, 226, 10)]
df_train[cond].plot(kind='line', x='layer_4', y=y_cols, figsize=(20, 5))